<a href="https://colab.research.google.com/github/Waseem771/Video-Based-Heart-Rate-and-Wellness-Monitoring/blob/main/Video-Based%20Heart%20Rate%20and%20Wellness%20Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless scipy matplotlib

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
import cv2
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import gradio as gr

def extract_forehead_region(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        return None

    # Use the first detected face
    (x, y, w, h) = faces[0]

    # Define forehead region (upper portion of the face)
    forehead = frame[y:y + int(0.2 * h), x:x + w]

    return forehead

def calculate_hrv(peaks):
    # Calculate RR intervals (time between heartbeats)
    rr_intervals = np.diff(peaks)

    # Calculate the standard deviation of RR intervals (SDNN)
    hrv = np.std(rr_intervals)

    return hrv

def estimate_respiratory_rate(signal_buffer, fps=30):
    # Detect slower periodic changes in the signal which may correspond to breathing
    respiration_peaks, _ = find_peaks(signal_buffer, distance=fps * 2)  # Assuming a lower frequency for breathing
    if len(respiration_peaks) > 1:
        respiratory_rate = len(respiration_peaks) * 60.0 / (len(signal_buffer) / fps)
        return respiratory_rate
    else:
        return None

def estimate_spo2(signal_buffer):
    # Very rough and experimental method to estimate SpO2
    green_channel = signal_buffer[:, 1]
    red_channel = signal_buffer[:, 2]
    spo2_estimation = 100 - 5 * (np.mean(red_channel) / np.mean(green_channel))
    return spo2_estimation

def get_heart_rate_from_video(video):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(video)

    # Parameters
    buffer_size = 256
    bpm_list = []

    # Buffers for storing the color values
    color_buffer = []
    signal_buffer = []

    face_detected = False  # Flag to check if face is detected

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        forehead = extract_forehead_region(frame, face_cascade)
        if forehead is not None:
            face_detected = True
            avg_color = np.mean(forehead, axis=(0, 1))
            color_buffer.append(avg_color)

        if len(color_buffer) >= buffer_size:
            # Process the buffer to find the heart rate
            color_buffer = np.array(color_buffer)
            green_channel = color_buffer[:, 1]

            # Remove the DC component by subtracting the mean
            green_channel = green_channel - np.mean(green_channel)

            # Store signal for plotting
            signal_buffer.append(green_channel)

            # Find peaks in the green channel signal
            peaks, _ = find_peaks(green_channel, distance=15)

            if len(peaks) > 1:
                # Calculate time between peaks (in frames)
                peak_intervals = np.diff(peaks)
                avg_interval = np.mean(peak_intervals)

                # Convert frame interval to time (assuming 30 FPS)
                avg_interval_sec = avg_interval / 30.0

                # Calculate BPM
                bpm = 60.0 / avg_interval_sec
                bpm_list.append(bpm)

                # Calculate HRV (Heart Rate Variability)
                hrv = calculate_hrv(peaks)

            color_buffer = []

    cap.release()

    if not face_detected:
        return {"Error": "Video format is not valid."}, None

    results = {}

    if bpm_list:
        avg_bpm = np.mean(bpm_list)
        adjusted_bpm = avg_bpm - 15
        results['Heart Rate'] = f"{adjusted_bpm:.2f} BPM"
    else:
        results['Heart Rate'] = "No heart rate detected."

    # Estimate Blood Pressure
    systolic_bp = 120 + 0.5 * (avg_bpm - 60)  # Simplified formula, not accurate
    diastolic_bp = 80 + 0.3 * (avg_bpm - 60)  # Simplified formula, not accurate

    results['Blood Pressure'] = f"{systolic_bp:.2f}/{diastolic_bp:.2f} mmHg"

    # Estimate Body Temperature (not accurate)
    if avg_bpm > 100:
        body_temperature = 37.0 + 0.1 * (avg_bpm - 100)  # Assuming fever if BPM is over 100
    else:
        body_temperature = 36.5  # Normal temperature

    results['Body Temperature'] = f"{body_temperature:.2f} °C"

    # Estimate Respiratory Rate
    if signal_buffer:
        respiration_signal = np.concatenate(signal_buffer)
        respiratory_rate = estimate_respiratory_rate(respiration_signal)
        if respiratory_rate:
            results['Respiratory Rate'] = f"{respiratory_rate:.2f} breaths/min"
        else:
            results['Respiratory Rate'] = "Unable to estimate Respiratory Rate."

    # Estimate SpO2 (not accurate)
    if color_buffer:
        spo2_estimation = estimate_spo2(np.array(color_buffer))
        results['SpO2'] = f"{spo2_estimation:.2f}%"

    # Basic Fatigue/Alertness Indicator
    if adjusted_bpm < 60:
        results['Fatigue/Alertness'] = "Possible fatigue detected."
    elif adjusted_bpm > 100:
        results['Fatigue/Alertness'] = "Possible alertness or stress detected."
    else:
        results['Fatigue/Alertness'] = "Normal state detected."

    # Plot the signal and save it
    if signal_buffer:
        plt.figure(figsize=(12, 6))
        signal_array = np.concatenate(signal_buffer)
        plt.plot(signal_array, label='Green Channel Signal')
        plt.xlabel('Frame')
        plt.ylabel('Signal Intensity')
        plt.title('Green Channel Signal from Forehead')
        plt.legend()
        plt.savefig("signal_plot.png")  # Save the plot as an image
        plt.close()  # Close the plot to prevent it from displaying inline

        return results, "signal_plot.png"

    return results, None

# Gradio Interface
def process_video(video):
    results, plot_path = get_heart_rate_from_video(video)
    return results, plot_path

# Interface setup
objective = "<b>Objective:</b> To develop a Gradio-based application that estimates heart rate, blood pressure, respiratory rate, body temperature, SpO2, and fatigue/alertness levels from facial video analysis, providing comprehensive vital sign monitoring."

team_members = """
<p>Team Members:</p>
<ul>
    <li>Irfan Jamshed (ID#2697) - engr.irfan@must.edu.pk</li>
    <li>Adnan Munir (ID#2031) - adnanmunir41@yahoo.com</li>
    <li>Muhammad Hannan Rauf (ID#2421) - hananrauf1@gmail.com</li>
    <li>Waseem Hassan (ID#2807) - engr.waseem77@gmail.com</li>
    <li>Dua Javed (ID#647) - duaajaved321@gmail.com</li>
</ul>
"""

iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(),
    outputs=[gr.JSON(), gr.Image()],
    title="HeartSense: Video-Based Heart Rate and Wellness Monitoring Using AI",
    description=f"{objective}<br>{team_members}",
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d57cacbfaead7d65e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
